Copyright (c) 2025 Christian Oechler

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE. 

## Importing the necessary python packages

In [1]:
import json
from pathlib import Path
import shutil
from tqdm.notebook import tqdm
import hashlib as hash

## Schritt 1: Generiert die Datei-Namen und speichert diese in eine Datei

In [ ]:
CLEAN_JSON_INPUTFILE = "data_clean.json"
CLEAN_PATH_OUTPUTFILE = "path.json"

In [ ]:
with open(CLEAN_JSON_INPUTFILE, 'r') as json_file:
    video_list = json.load(json_file)

data_finished = []

with tqdm(video_list.keys(), desc="Fortschritt", unit="Datei") as pbar:
    for i, key in enumerate(pbar):
        video_metadata = video_list[key]
        source_file_path = str(key)

        video_id = video_metadata["video_id"]
        video_date = video_metadata["video_date"]
        video_camera = video_metadata["video_camera"]
        video_time = video_metadata["video_time"]
    
        video_date_elements = video_date.split("-")
        day = video_date_elements[0]
        month = video_date_elements[1]
        year = video_date_elements[2]

        video_time = video_time.replace(":","")

        filepath = f'{video_camera}/{year}/{month}/{day}'
        filesuffix = Path(source_file_path).suffix
        filename = f'{video_id}_{video_camera}_{video_time}{filesuffix}'

        full_path = f'{filepath}/{filename}'

        video_data = {"source_file_path": source_file_path, "destination_file_path": full_path}

        data_finished.append(video_data)

with open(CLEAN_PATH_OUTPUTFILE, 'w') as json_file:
    json.dump(data_finished, json_file, indent=4)        

## Schritt 2: Dateien kopieren

In [3]:
CLEAN_PATH_INPUTFILE = "path.json"
DESTINATION_DIRECTORY = "renamed_videos"

In [4]:
def get_sha256_checksum(filepath: str) -> str:
    BLOCKSIZE = 65536
    
    sha = hash.sha256()
    
    with open(filepath, 'rb') as handler:
        file_buffer = handler.read(BLOCKSIZE)
            
        while len(file_buffer) > 0:
            sha.update(file_buffer)
            file_buffer = handler.read(BLOCKSIZE)
            
    return sha.hexdigest()

In [5]:
def copy_videofiles(source_file_path, destination_file_path, attempts=5) -> bool:
    for _ in range(attempts):
        shutil.copy2(source_file_path, destination_file_path)

        # Calculates the checksum of the source and destination file
        checksum_source_file = get_sha256_checksum(f'{source_file_path}')
        checksum_destination_file = get_sha256_checksum(f'{destination_file_path}')

        # Checks that the file was successfully copied
        if checksum_source_file == checksum_destination_file:
            return True

    return False

In [21]:
with open(CLEAN_PATH_INPUTFILE, 'r') as json_file:
    data = json.load(json_file)

with tqdm(data, desc="Fortschritt", unit="Datei") as pbar:
    for i, file in enumerate(pbar):
        try:
            source_file_path = Path(file["source_file_path"])
            destination_file_path = Path(file["destination_file_path"])
            
            # Checks the existance of the source video file
            if not source_file_path.exists() or not source_file_path.is_file():
                print(f'Die Videodatei {source_file_path} existiert nicht.')
                continue
    
            destination_directory = Path(DESTINATION_DIRECTORY) / destination_file_path.parent
            destination_directory.mkdir(parents=True, exist_ok=True)
    
            pbar.set_postfix_str(f'{source_file_path.name} => {destination_file_path}')
    
            final_destination = Path(DESTINATION_DIRECTORY) / destination_file_path
            
            result = copy_videofiles(source_file_path, final_destination)

            if not result:
                raise Exception

        except Exception:
            print(f'Es ist ein Fehler beim Kopieren der Datei "{file["source_file_path"]}" aufgetreten.')
            continue

Fortschritt:   0%|          | 0/125 [00:00<?, ?Datei/s]